<a href="https://colab.research.google.com/github/HwangHanJae/BigDataPratice/blob/main/6_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#작업형1

데이터 출처 :https://www.kaggle.com/mohansacharya/graduate-admissions(참고, 데이터 수정)

데이터 설명 : 대학원 입학 가능성 예측

Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv

##Q1. Serial No. 컬럼을 제외하고 'Chance of Admit'을 종속변수, 나머지 변수를 독립변수라 할때, 랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)

In [13]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")

X = df.drop(["Serial No.", "Chance of Admit"], axis=1)
y = df["Chance of Admit"]

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf.fit(X, y)
feature_important = rf.feature_importances_

temp = pd.DataFrame({"feature" : rf.feature_importances_}, index= X.columns)
temp.sort_values("feature", ascending = False)

,feature
CGPA,0.820562
GRE Score,0.079885
TOEFL Score,0.029059
SOP,0.024695
LOR,0.024523
University Rating,0.013372
Research,0.007904


#작업형2


데이터 출처 :https://www.kaggle.com/mohansacharya/graduate-admissions(참고, 데이터 수정)

데이터 설명 : 대학원 입학 가능성 예측문제 ("Chance of Admit')

​평가지표 : r2 score

trainData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv

testData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/test.csv


In [19]:
trainData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv"
testData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/test.csv"

import pandas as pd

train = pd.read_csv(trainData)
test= pd.read_csv(testData)

train

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,246,328,110,4,4.0,2.5,9.02,1,0.81
1,100,323,113,3,4.0,4.0,8.88,1,0.79
2,79,296,95,2,3.0,2.0,7.54,1,0.44
3,53,334,116,4,4.0,3.0,8.00,1,0.78
4,444,321,114,5,4.5,4.5,9.16,1,0.87
...,...,...,...,...,...,...,...,...,...
395,47,329,114,5,4.0,5.0,9.30,1,0.86
396,14,307,109,3,4.0,3.0,8.00,1,0.62
397,28,298,98,2,1.5,2.5,7.50,1,0.44
398,149,339,116,4,4.0,3.5,9.80,1,0.96


In [20]:
test

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,401,304,100,2,3.5,3.0,8.22,0
1,402,315,105,2,3.0,3.0,8.34,0
2,403,324,109,3,3.5,3.0,8.94,1
3,404,330,116,4,4.0,3.5,9.23,1
4,405,311,101,3,2.0,2.5,7.64,1
...,...,...,...,...,...,...,...,...
95,496,332,108,5,4.5,4.0,9.02,1
96,497,337,117,5,5.0,5.0,9.87,1
97,498,330,120,5,4.5,5.0,9.56,1
98,499,312,103,4,4.0,5.0,8.43,0


In [21]:
#필요없는 컬럼 제거

train = train.drop("Serial No.", axis=1)
test = test.drop("Serial No.", axis=1)

#타겟데이터 분리
X = train.drop("Chance of Admit", axis=1)
y = train["Chance of Admit"]

X.shape, test.shape

((400, 7), (100, 7))

In [26]:
#훈련데이터, 검증데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val  = train_test_split(X, y, test_size=0.33, random_state=42)


#모델링 & 평가

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import r2_score

rf = RandomForestRegressor(random_state=42)
xgb =XGBRegressor(random_state=42)
lgbm = LGBMRegressor(random_state=42)
tree = DecisionTreeRegressor(random_state=42)
knn = KNeighborsRegressor()


models = [rf, xgb, lgbm, tree, knn]

for model in models:
  model.fit(X_train, y_train)
  print(model.__class__.__name__)

  y_pred = model.predict(X_val)
  print(r2_score(y_val, y_pred))



RandomForestRegressor
0.8184190779939806
[05:35:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor
0.8153809741150196
LGBMRegressor
0.8125895665493288
DecisionTreeRegressor
0.6438064332338334
KNeighborsRegressor
0.7402261664416563


In [29]:
#제출
import numpy as np

sub = {"Chance of Admit" : np.zeros(test.shape[0])}
pred = rf.predict(test)
submission = pd.DataFrame(sub)

submission["Chance of Admit"] = pred
submission.to_csv("수험번호.csv", index=False)